In [6]:
import numpy as np
import pandas as pd

from vk import VkAPI
import multiprocessing as mp
from time import sleep
from multiprocessing import Process, Pool

### Import list of users ids

In [2]:
df = pd.read_csv('users_activities_groups.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates(subset=['user_id'])

In [3]:
users = list(df['user_id'])
len(users)

28997

### Collect music

In [4]:
vk_api = VkAPI()

In [16]:
def download_music_for_users(users, filename_music='music', filename_errors='errors', percent=0.5):
    for user_id in users:
        try:
            tracks_list = vk_api.get_music_list(user_id=user_id)
            for track in tracks_list:
                with open(filename_music, 'a', encoding='utf-8') as tracks_file:
                    tracks_file.write(f'{user_id},{track["id"]},{track["artist"]},{track["title"]}\n')
        except Exception as ex:
            with open(filename_errors, 'a', encoding='utf-8') as errors_file:
                errors_file.write(f'{user_id},{ex}\n')

In [18]:
def run(users, processes_num=8):

    users_batch_size = len(users) // processes_num
    users_batches = []
    j = users_batch_size
    for k in range(processes_num):
        i = users_batch_size * k
        users_batches.append(users[i:j])
        j = users_batch_size * (k + 2)

    pool = Pool(processes_num)
    pool.map(download_music_for_users, [row for row in users_batches])
    pool.close()

In [ ]:
run(users)